Further synonym mapping pipeline experiments

In [ ]:
import os
import json
from dotenv import load_dotenv
import anthropic

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)


In [5]:
# set path for testing
import sys
import os
parent_dir = os.path.abspath("../../")
print(parent_dir)
sys.path.append(parent_dir)

from embeddings import snomed_store, loinc_store
from embeddings.embeddings import SearchResult

snomed_store = snomed_store.connect_snomed()
loinc_store = loinc_store.connect_loinc()

/Users/hanna/openfn/ai_experiments/apollo/services


/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Synonym approach 2) Guess & Search

This approach aims to address the problems noted above the first method (when input texts do not look at all like the output). Leverage the LLM's in-built intuition for clinical terminology.

Step 1 - Use LLM to output the LOINC text
Step 2 - Use the text from the LLM output to search for the entry. (option b if we see issues: use both original query + the LLM output to search the db)

In [7]:
# import data to temporarily plug in missing metadata that will be in the vector store
import pandas as pd

df = pd.read_csv("/Users/hanna/openfn/ai_experiments/data/LOINC-Clinical-Terminology/LoincTableCore.csv")
loinc_num_dict = dict(zip(df.LONG_COMMON_NAME.to_list(), df.LOINC_NUM.to_list()))

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/1852639281.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/hanna/openfn/ai_experiments/data/LOINC-Clinical-Terminology/LoincTableCore.csv")


In [240]:
# use LLM to get input text CT synonym then search vector store. Also add hybrid search.

synonym_expand_system_prompt = """
You are an assistant for matching terminology between health record systems. 
You will be given a source text from one health record system, which might be in another language. 
You should output up to 20 of the most probable equivalent names (the long name, not the code) in LOINC clinical terminology with no further explanation.
The terms should be general and non-specific, unless additional information has been given. 
Add each result on a new line without numbering.
"""

synonym_expand_user_prompt = """
Source text to output a LOINC term for: "{input_text}"
"""

synonym_expand_select_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given a LOINC entry to look for in a list of similar records. You will also be given the original source term for context.
Choose the most general entry that reflects the original source term accurately unless additional information has been given. Favour short, generic entries over those that imply specific external systems or methods.
You should select the correct LOINC entry from the options and output just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon.
"""

synonym_expand_select_user_prompt = """
Original source term for context: {input_text}
LOINC entry to look for: {LLM_guess}
LOINC entries to select from: "{retreived_texts}"
"""

def synonym_mapping(input_query):
    # input_query = "malaria_RDT_result"

    synonym_expand_user_prompt_formatted = synonym_expand_user_prompt.format(input_text=input_query)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_answer = message.content[0].text
    print(f"LLM guess for the CT: {synonym_expand_answer}")

    # split guesses and search for each guess:
    guesses = synonym_expand_answer.split("\n")
    synonym_expand_retreived_data = []
    for guess in guesses:
        guess_search_results = loinc_store.search(guess, search_kwargs={"k": 10})
        synonym_expand_retreived_data.extend(guess_search_results)

    # synonym_expand_retreived_data = loinc_store.search(input_query, search_kwargs={"k": 100})

    # add keyword search to conduct hybrid search
    keyword_search_results = []
    for guess in guesses:
        guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()
        # some words like "weight" might have too many results and max out our prompt. Keep just the shortest answers.
        if len(guess_keyword_search_results) > 100:
            guess_keyword_search_results = sorted(guess_keyword_search_results, key=len)[:100]
        guess_keyword_search_results = [SearchResult(text=json.dumps({"LONG_COMMON_NAME": s}), metadata={}, score=None) for s in guess_keyword_search_results]
        keyword_search_results.extend(guess_keyword_search_results)
    
    # add loinc codes to the keyword search results
    keyword_search_results = [SearchResult(text=json.dumps({'LOINC_NUM': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME']), **json.loads(r.text)}), metadata=r.metadata, score=r.score) for r in keyword_search_results]

    search_results_combined = keyword_search_results+synonym_expand_retreived_data

    # temp fix for missing loinc numbers
    # search_results_combined = [SearchResult(text=r.text, metadata={**r.metadata, 'LOINC_NUMBER': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME'])}, score=r.score) for r in search_results_combined]
    
    with open("search_results_tmp.txt", "w") as f:
        f.writelines("\n".join([str(s) for s in search_results_combined]))
    # tried this rule-based retreival, but there are unexpected issues e.g. "Body height" hits "Body height Mother"
    # final_search_result = ""
    # for search_result in synonym_expand_retreived_data:
    #     if synonym_expand_answer in search_result.metadata["LONG_COMMON_NAME"]:
    #         final_search_result = (search_result, loinc_num_dict[search_result.metadata["LONG_COMMON_NAME"]])
    # final_search_result

    synonym_expand_select_user_prompt_formatted = synonym_expand_select_user_prompt.format(input_text=input_query, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_select_answer = message.content[0].text
    
    return synonym_expand_select_answer

In [152]:
synonym_mapping("cervical_cancer_result")

LLM guess for the CT: Cancer of cervix
Cervical cancer screening
Cervical cytology
Cervical Pap smear findings
Cervical cancer diagnosis
Cervical cancer status
Cervical intraepithelial neoplasia
Cervical cancer screening interpretation
Cervical cancer risk assessment
Cervical cancer screening results
Cervical cancer screening status
Cervical cancer screening findings
Cervical cancer screening outcome
Cervical cancer screening test
Cervical cancer screening evaluation
Cervical cancer screening report
Cervical cancer screening results narrative
Cervical cancer screening interpretation narrative
Cervical cancer screening assessment
Cervical cancer screening conclusion


'Based on the source term "cervical_cancer_result" and looking for "Cancer of cervix" in the LOINC entries, I\'ll select the most general and appropriate entry that reflects cancer screening/results without specifying a particular method.\n\nThe most appropriate match would be:\n\nHEDIS 2009-2013 Codes to identify cervical cancer screening tests (CCS-A);54038-5\n\nThis entry is:\n1. Specifically about cervical cancer screening tests\n2. Does not specify a particular testing method\n3. Is designed to capture general cervical cancer screening results\n4. Matches the general nature of the source term "cervical_cancer_result"'

In [30]:
# test data

fields = [
   'systolic_blood_pressure_mmhg',
   'diastolic_blood_pressure_mmhg',
   'temperature',
   'pulse_bpm',
   'respiration_rate_rpm',
   'avpu_score',
   'height',
   'weight',
   'head',
   'eyes',
   'ear_nose_throat',
   'has_the_patient_ever_had_a_genital_infection',
   'abdomen',
   'heart',
   'cholesterol_test_results',
   'cervical_cancer_result',
   'hemoglobin_result',
   'hepatitis_b_result',
   'hepatitis_c_result',
   'hiv_result',
   'tuberculosis_test_result',
   'urinalysis_result',
   'urinary_hcg_result',
   'gdp',
   'gd2pp',
   'gds_result',
   'asam_urat_results',
   'malaria_RDT_result',
   'malaria_RDT_test',
   'igg_result',
   'igm_result',
   'bhcg_result',
   'how_many_pregnancies_have_they_had',
   'how_many_pregnancies_have_they_aborted_or_miscarried',
   'first_day_of_last_menstrual_period',
   'estimated_date_of_delivery',
   'bmi',
   'muac',
   'smoking_history',
   'how_many_alcoholic_drinks_per_week',
   'extremities_arm_leg',
   'musculoskeletal',
   'neurological',
   'skin',
   'lungs',
   'uterine_height_cm',
   'syphilis_result',
   'blood_oxygen_satutation',
   'current_contraceptive',
   'non-prescribed_drug_use',
   'how_many_children',
   'extraoral'
]

In [29]:
gold_labels = [
'Systolic blood pressure;8480-6',
'Diastolic blood pressure;8462-4',
'Body temperature;8310-5',
'Heart rate;8867-4',
'Respiratory rate;9279-1',
'Level of responsiveness;67775-7',
'Body height;8302-2',
'Body weight;29463-7',
'Physical findings of Head Narrative;10199-8',
'Physical findings of Eye Narrative;10199-8',
'Ear, nose and throat finding;297268004',
'Physical findings of Genitalia Narrative;10199-8',
'Physical findings of Abdomen Narrative;10191-5',
'Physical findings of Heart Narrative;10200-4',
'Cholesterol Mass/volume in Serum or Plasma;2093-3',
'Microscopic observation Identifier in Cervical or vaginal smear or scraping by Cyto stain;2093-3',
'Hemoglobin Mass/volume in Blood;718-7',
'Hepatitis B virus surface Ag [Presence] in Serum, Plasma or Blood by Rapid immunoassay;75410-1',
'Hepatitis C virus Ab Presence in Serum;16128-1',
'HIV 1 Ab [Presence] in Serum, Plasma or Blood by Rapid immunoassay;68961-2',
'Microscopic observation [Identifier] in Sputum by Acid fast stain;11477-7',
'Urinalysis complete panel - Urine;24356-8',
'Choriogonadotropin.beta subunit (pregnancy test) Presence in Urine;2112-1',
'Fasting glucose Mass/volume in Serum or Plasma;1558-6',
'Glucose Mass/volume in Serum or Plasma --2 hours post meal;1521-4',
'Glucose [Mass/volume] in Capillary blood by Glucometer;41653-7',
'Urate Mass/volume in Serum or Plasma;3084-1',
'Plasmodium sp Ag Identifier in Blood by Rapid immunoassay;70569-9',
'Plasmodium sp Ag Identifier in Blood by Rapid immunoassay;70569-9',
'IgG Mass/volume in Serum or Plasma;2465-3',
'IgM Mass/volume in Serum or Plasma;3182-3',
'Choriogonadotropin.beta subunit Units/volume in Serum or Plasma;21198-7',
'# Pregnancies;11996-6',
'Other pregnancy outcomes #;69043-8',
'Last menstrual period start date;8665-2',
'Delivery date Estimated;11778-8',
'Body mass index (BMI) Ratio;39156-5',
'Mid upper arm circumference;284473002',
'Tobacco smoking status;72166-2',
'History of Alcohol use;11331-6',
'Physical findings of Extremities Narrative;10196-4',
'Physical findings of Musculoskeletal system Narrative;11410-8',
'Physical findings of Neurologic balance and Coordination;8712-2',
'Physical findings of Skin Narrative;10206-1',
'Physical findings of Lung;32449-1',
'Uterus Fundal height Tape measure;11881-0',
'Treponema pallidum Ab [Presence] in Serum by Hemagglutination;8041-6',
'Oxygen saturation in Arterial blood;2708-6',
'Birth control method at intake Reported;86649-1',
'I took medication that had not been prescribed or if had been prescribed, I took more than the prescribed dose;72639-8',
'[#] Births.still living;11638-4',
'Oral evaluation exam;34045-5'
]

In [47]:
print(len(fields))
len(gold_labels)

52


52

In [ ]:
# v2
results_f = "results_synonym_prompt_v2_run2.csv"
output_eval_f = "results_synonym_prompt_v2_run2_evaluated.csv"

results = []
for field, gold_label in zip(fields, gold_labels):
    result = synonym_mapping(field)
    results.append([field, gold_label, result])
results_df = pd.DataFrame(results, columns=["Input field", "Correct answer", "System answer"])
results_df.to_csv(results_f, index=False)

# assess selected responses for exact matches
answers = []
for gold, prediction in zip(results_df["Correct answer"].to_list(), results_df["System answer"].to_list()):
    answer = assess_response(gold, prediction)
    answers.append(answer)
results_df["Exactly same code as in the lookup-table?"] = answers
results_df["Exactly same code as in the lookup-table?"] = results_df["Exactly same code as in the lookup-table?"].map({'True': True, 'False': False})
answer_mean = results_df["Exactly same code as in the lookup-table?"].mean()
print(f"Exact success rate {answer_mean}")
results_df.to_csv(output_eval_f, index=False)
results_df["Exactly same code as in the lookup-table?"].value_counts()

LLM guess for the CT: Systolic blood pressure
Blood pressure systolic
Systolic blood pressure by device
Systolic blood pressure by automated reading
Systolic blood pressure by manual reading
Systolic blood pressure by cuff
Systolic blood pressure by automated cuff
Systolic blood pressure by manual cuff
Systolic blood pressure by auscultation
Systolic blood pressure by palpation
Systolic blood pressure by oscillometric method
Systolic blood pressure by Doppler
Systolic blood pressure by arterial line
Systolic blood pressure.peripheral
Systolic blood pressure - sitting
Systolic blood pressure - standing
Systolic blood pressure - supine
Systolic blood pressure - lying
Systolic blood pressure measurement
Blood pressure panel with all children optional
LLM guess for the CT: Blood pressure Diastolic
Diastolic blood pressure
Blood pressure.diastolic
Diastolic blood pressure by Automated oscillometric method
Diastolic blood pressure by Manual auscultation
Diastolic blood pressure by Automated 

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2137567002.py:64: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Chorionic gonadotropin.beta subunit [Units/volume] in Urine
Chorionic gonadotropin [Units/volume] in Urine
Pregnancy test
Pregnancy test - urine
Chorionic gonadotropin [Presence] in Urine
Chorionic gonadotropin.beta subunit [Presence] in Urine
Human chorionic gonadotropin [Presence] in Urine
Human chorionic gonadotropin [Units/volume] in Urine
Pregnancy test - qualitative
Pregnancy test - quantitative
LLM guess for the CT: Glucose [Mass/volume] in Blood
Glucose [Moles/volume] in Blood
Glucose [Mass/volume] in Serum or Plasma
Glucose [Moles/volume] in Serum or Plasma
Glucose [Mass/volume] in Urine
Glucose [Presence] in Urine
Glucose [Mass/volume] in Cerebral spinal fluid
Glucose [Mass/volume] in Body fluid
Glucose [Mass/volume] in Blood by Glucometer
Glucose [Mass/volume] in Capillary blood
LLM guess for the CT: Glucose [Mass/volume] in Blood
Glucose [Moles/volume] in Blood
Glucose [Mass/volume] in Serum or Plasma
Glucose [Moles/volume] in Serum or Plasma
Glucose [

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2137567002.py:64: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Mid-upper arm circumference
Arm circumference
Upper arm circumference
MUAC measurement
Arm circumference by tape measure
Body part circumference
Circumference.mid-upper arm
Arm circumference measurement
Extremity circumference
Body circumference measurement
LLM guess for the CT: Tobacco smoking status
History of tobacco use
Tobacco use
Smoking status
History of smoking
Tobacco use status
Smoking history
Current smoking status
Tobacco use pattern
Tobacco smoking behavior
Smoking behavior pattern
Tobacco use finding
Smoking assessment
Tobacco use assessment
History of tobacco smoking
LLM guess for the CT: Alcoholic drinks per week
Alcohol intake
Alcohol consumption
Alcohol drinking behavior
Alcoholic beverages consumed per week
Frequency of alcohol consumption
Alcohol use pattern
Alcohol use frequency
Number of alcoholic drinks per week
Weekly alcohol consumption
Alcohol intake pattern
Drinking pattern
Alcohol use assessment
Alcohol consumption frequency
Alcoholic b

In [242]:
results_df.head()

,Input field,Correct answer,System answer
0,systolic_blood_pressure_mmhg,Systolic blood pressure;8480-6,Systolic blood pressure;8480-6
1,diastolic_blood_pressure_mmhg,Diastolic blood pressure;8462-4,Diastolic blood pressure by Noninvasive;76535-4
2,temperature,Body temperature;8310-5,Body temperature;8310-5
3,pulse_bpm,Heart rate;8867-4,Heart rate;8867-4
4,respiration_rate_rpm,Respiratory rate;9279-1,Respiratory rate;9279-1\n\nI chose this entry ...


In [ ]:
# V3 Synonym mapping - Add USER INPUT fields for the dataset and each entry; shortlist + top 1 selections

# Guess probable entry names
synonym_expand_system_prompt = """
You are an assistant for matching terminology between health record systems. 
You will be given a source text from one health record system, which might be in another language. 
Output up to 20 of the most probable equivalent LOINC clinical terminology names (long names only, no codes). 
Only include terms that match the specificity level of the input - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
Add each result on a new line without numbering.
"""

synonym_expand_user_prompt = """
{general_info}

Source text for which to output LOINC terms: "{input_text}"
{specific_info}
"""

# Select top entries from search results
# synonym_expand_select_system_prompt = """
# You are an assisstant for matching terminology between health record systems. 
# You will be given an entry from a source health record system, which you need to map to the most accurate LOINC term. You will also be given a list of the probable entry names to look out for. Additional contextual information may also be given.
# Choose 10 of the most general entries that reflect the original source term precisely. If no specific methods or external systems are specified in the input, favour broad entries that can capture different types of scenarios.
# Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon.
# """
synonym_expand_select_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given an entry from a source health record system, which you need to map to the most accurate LOINC term. To help you, you will also be given a list of the probable entry names to look out for. Additional contextual information may also be given.
Choose 10 entries that reflect the original source term best. Only include terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon.
"""

synonym_expand_select_user_prompt = """
{general_info}
Original source term for which to look for a LOINC entry: {input_text}
{specific_info}
The target LOINC entry text might look like one of these: {LLM_guess}
LOINC entries to select from: "{retreived_texts}"
"""

# Select final entry from shortlist
synonym_expand_select_top_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given an entry from a source health record system, which you need to map to the most accurate LOINC term from a given list of options.
You may also be given additional contextual information.
Choose the most general entry that reflects the original source term precisely. If no specific methods or external systems are specified in the input, favour broad entries that can capture different types of scenarios.
Give your response as just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon.
"""

synonym_expand_select_top_user_prompt = """
{general_info}
Original source term for which to look for a LOINC entry: {input_text}
{specific_info}
LOINC entries to select from: "{term_shortlist}"
"""

def synonym_mapping_with_user_input(input_query, user_general="", user_specific=""):
    """"""
    if user_general:
        user_general = "Additional general information about the dataset: " + user_general
    if user_specific:
        user_specific = "Additional information about this input: " + user_specific
        
    synonym_expand_user_prompt_formatted = synonym_expand_user_prompt.format(input_text=input_query, general_info=user_general, specific_info=user_specific)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_answer = message.content[0].text
    print(f"LLM guess for the CT: {synonym_expand_answer}")

    # split guesses and search for each guess:
    guesses = synonym_expand_answer.split("\n")
    synonym_expand_retreived_data = []
    for guess in guesses:
        guess_search_results = loinc_store.search(guess, search_kwargs={"k": 10})
        synonym_expand_retreived_data.extend(guess_search_results)

    # add keyword search to conduct hybrid search
    keyword_search_results = []
    for guess in guesses:
        guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()
        # some words like "weight" might have too many results and max out our prompt. Keep just the shortest answers.
        if len(guess_keyword_search_results) > 100:
            guess_keyword_search_results = sorted(guess_keyword_search_results, key=len)[:100]
        guess_keyword_search_results = [SearchResult(text=json.dumps({"LONG_COMMON_NAME": s}), metadata={}, score=None) for s in guess_keyword_search_results]
        keyword_search_results.extend(guess_keyword_search_results)
    
    # add loinc codes to the keyword search results
    keyword_search_results = [SearchResult(text=json.dumps({'LOINC_NUM': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME']), **json.loads(r.text)}), metadata=r.metadata, score=r.score) for r in keyword_search_results]

    search_results_combined = keyword_search_results+synonym_expand_retreived_data
    
    with open("search_results_tmp.txt", "w") as f:
        f.writelines("\n".join([str(s) for s in search_results_combined]))


    # Ask LLM to select a shortlist of the best terms
    synonym_expand_select_user_prompt_formatted = synonym_expand_select_user_prompt.format(general_info=user_general, input_text=input_query, specific_info=user_specific, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    term_shortlist = message.content[0].text

    # Ask LLM to select the best entry from the shortlist
    synonym_expand_select_top_user_prompt_formatted = synonym_expand_select_top_user_prompt.format(general_info=user_general, input_text=input_query, specific_info=user_specific, term_shortlist=term_shortlist)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_top_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_top_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    top_term = message.content[0].text
    
    return (top_term, term_shortlist)

In [230]:
synonym_mapping_with_user_input("hepatitis_b_result")

LLM guess for the CT: Hepatitis B virus surface Ag
Hepatitis B virus core Ab
Hepatitis B virus surface Ab
Hepatitis B virus DNA
Hepatitis B virus core Ab IgM
Hepatitis B virus surface Ab response
Hepatitis B virus e Ag
Hepatitis B virus e Ab
Hepatitis B virus immunity status
Hepatitis B virus serologic markers panel
Hepatitis B virus infection status
Hepatitis B virus interpretation
Hepatitis B virus genotype
Hepatitis B virus resistance
Hepatitis B virus risk factors


('Hepatitis B virus Ab [Interpretation] in Serum or Plasma;95235-8',
 'Hepatitis B virus surface Ag [Presence] in Serum or Plasma by Immunoassay;5196-1\nHepatitis B virus core Ab [Presence] in Serum or Plasma by Immunoassay;13952-7\nHepatitis B virus surface Ab [Units/volume] in Serum or Plasma by Immunoassay;5193-8\nHepatitis B virus DNA [Presence] in Serum or Plasma by NAA with probe detection;29610-3\nHepatitis B virus core IgM Ab [Presence] in Serum or Plasma by Immunoassay;24113-3\nHepatitis B virus surface Ab [Presence] in Serum;22322-2\nHepatitis B virus e Ag [Presence] in Serum or Plasma by Immunoassay;13954-3\nHepatitis B virus e Ab [Presence] in Serum or Plasma by Immunoassay;13953-5\nHepatitis B virus Ab [Interpretation] in Serum or Plasma;95235-8\nHepatitis B virus core Ab and surface and little e Ab and Ag panel - Serum or Plasma;92898-6')

In [ ]:
synonym_mapping_with_user_input("how_many_children", user_general="", user_specific="Select a code that refers to how many living children the patient has, not how many live in their household")

LLM guess for the CT: Number of children
Number of live births
Parity
Number of children living
Births.live
Gravida
Para
Number of offspring
Number of children born alive
Living children count
Number of biological children
Live births total
Parity living
Children living
Births live total


("Mother's Births.live # [Reported];75202-2",
 "[#] Births.live;11636-8\nMother's Births.live # [Reported];75202-2\n[#] Parity;11977-6\nNumber of children under the age of 18 living in mother's household during the pregnancy;85722-7\nNumber of children under the age of 14 living in household;98155-5\nHow many children under the age of 18 live in your household # [SAMHSA];68508-1\nNumber of infants in this delivery delivered alive;73773-4\nInfant living at time of report [US Standard Certificate of Live Birth];73757-7\nHow many people are living or staying at this address [#];63512-8\nHow many of your children have normal hearing [#] [PhenX];67419-2")

In [28]:
# assess if pair is the same

assess_system_prompt = """
You are an assistant for assessing whether two pairs of Clinical Terminology entries are exactly the same.
The pair is only similar if both the name and the code are the same in each entry.
ONLY output True or False with no further explanation.
"""

assess_prompt = """
Clinical terms to compare for similarity:
CT 1: "{gold}"
CT 2: "{prediction}"
"""

def assess_response(gold, prediction):
    """Use Claude to assess correct answer and prediction pair."""

    assess_prompt_formatted = assess_prompt.format(gold=gold, prediction=prediction)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=assess_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": assess_prompt_formatted
                    }
                ]
            }
        ]
    )
    assess_response = message.content[0].text

    return assess_response

In [238]:
results_df["Exactly same code as in the lookup-table?"].value_counts()

Exactly same code as in the lookup-table?
False    34
True     18
Name: count, dtype: int64

In [ ]:
#  V4(e) with shortlist + top 1 selection with FULL data; zero-shot with no user input; ONLY 15 starting guesses + ADD GENERAL CONTEXT INFO + Narrative tags

# Synonym mapping - Add USER INPUT fields for the dataset and each entry; shortlist + top 1 selections with REPEAT use of FULL SEARCH DATA 

# Guess probable entry names
synonym_expand_system_prompt = """
You are an assistant for matching terminology between health record systems. 
You will be given a source text from one health record system, which might be in another language. 
Output up to 15 of the most probable equivalent LOINC clinical terminology names (long names only, no codes). 
Only include terms that match the specificity level of the input - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
Add each result on a new line without numbering.
"""

synonym_expand_user_prompt = """
{general_info}

Source text for which to output LOINC terms: "{input_text}"
{specific_info}
"""

synonym_expand_select_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given an entry from a source health record system, and a list of possible target LOINC terms. Select 10 target LOINC terms that reflect the original source term best. 
To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon.
"""

synonym_expand_select_user_prompt = """
{general_info}
Original source term for which to look for a LOINC entry: {input_text}
{specific_info}
The correct target LOINC entry text is likely to look like one of these: {LLM_guess}
LOINC entries to select from: "{retreived_texts}"
"""

# Select final entry from FULL search results
synonym_expand_select_top_system_prompt = """
You are an assisstant for matching terminology between health record systems. 
You will be given an entry from a source health record system, and a list of possible target LOINC terms. Select just ONE target LOINC term that reflects the original source term best. 
To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
Give your response as just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon.
"""

synonym_expand_select_top_user_prompt = """
{general_info}
Original source term for which to look for a LOINC entry: {input_text}
{specific_info}
The correct target LOINC entry text is likely to look like one of these: {LLM_guess}
LOINC entries to select from: "{retreived_texts}"
"""

def synonym_mapping_with_user_input_repeat_search(input_query, user_general="", user_specific=""):
    """"""
    if user_general:
        user_general = "Additional general information about the dataset: " + user_general
    if user_specific:
        user_specific = "Additional information about this input: " + user_specific
        
    synonym_expand_user_prompt_formatted = synonym_expand_user_prompt.format(input_text=input_query, general_info=user_general, specific_info=user_specific)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    synonym_expand_answer = message.content[0].text
    print(f"LLM guess for the CT: {synonym_expand_answer}")

    # split guesses and search for each guess:
    guesses = synonym_expand_answer.split("\n")
    synonym_expand_retreived_data = []
    for guess in guesses:
        guess_search_results = loinc_store.search(guess, search_kwargs={"k": 10})
        synonym_expand_retreived_data.extend(guess_search_results)

    # add keyword search to conduct hybrid search
    keyword_search_results = []
    for guess in guesses:
        guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()
        # some words like "weight" might have too many results and max out our prompt. Keep just the shortest answers.
        if len(guess_keyword_search_results) > 100:
            guess_keyword_search_results = sorted(guess_keyword_search_results, key=len)[:100]
        guess_keyword_search_results = [SearchResult(text=json.dumps({"LONG_COMMON_NAME": s}), metadata={}, score=None) for s in guess_keyword_search_results]
        keyword_search_results.extend(guess_keyword_search_results)
    
    # add loinc codes to the keyword search results
    keyword_search_results = [SearchResult(text=json.dumps({'LOINC_NUM': loinc_num_dict.get(json.loads(r.text)['LONG_COMMON_NAME']), **json.loads(r.text)}), metadata=r.metadata, score=r.score) for r in keyword_search_results]

    search_results_combined = keyword_search_results+synonym_expand_retreived_data
    
    with open("search_results_tmp.txt", "w") as f:
        f.writelines("\n".join([str(s) for s in search_results_combined]))


    # Ask LLM to select a shortlist of the best terms
    synonym_expand_select_user_prompt_formatted = synonym_expand_select_user_prompt.format(general_info=user_general, input_text=input_query, specific_info=user_specific, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    term_shortlist = message.content[0].text

    # Ask LLM to select the best entry from the shortlist
    synonym_expand_select_top_user_prompt_formatted = synonym_expand_select_top_user_prompt.format(general_info=user_general, input_text=input_query, specific_info=user_specific, LLM_guess=synonym_expand_answer, retreived_texts=search_results_combined)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=synonym_expand_select_top_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": synonym_expand_select_top_user_prompt_formatted
                    }
                ]
            }
        ]
    )
    top_term = message.content[0].text
    
    return (top_term, term_shortlist)


output_f = "results_synonym_prompt_v3c_15startguess_withgeninfov1_withindivinfov1.csv"
output_eval_f = "results_synonym_prompt_v3c_evaluated_15startguess_withgeninfov1_withindivinfov1.csv"
output_shortlist_eval_f = "results_synonym_prompt_v3c_evaluated_shortlist_15startguess_withgeninfov1_withindivinfov1.csv"
user_general_context = "The input term is from the Indonesian Satusehat patient healthcare records system."

# run LLM on dataset
results = []
for field, gold_label in zip(fields, gold_labels):
    if "narrative" in gold_label.lower():
        result = synonym_mapping_with_user_input_repeat_search(input_query=field, user_general=user_general_context, user_specific="This is a doctor's narrative field")
    else:
        result = synonym_mapping_with_user_input_repeat_search(input_query=field, user_general=user_general_context)
    results.append([field, gold_label, result[1], result[0]])
results_df = pd.DataFrame(results, columns=["Input field", "Correct answer", "System shortlist", "System answer"])
results_df.to_csv(output_f, index=False)

# assess selected responses for exact matches
answers = []
for gold, prediction in zip(results_df["Correct answer"].to_list(), results_df["System answer"].to_list()):
    answer = assess_response(gold, prediction)
    answers.append(answer)
results_df["Exactly same code as in the lookup-table?"] = answers
results_df["Exactly same code as in the lookup-table?"] = results_df["Exactly same code as in the lookup-table?"].map({'True': True, 'False': False})
answer_mean = results_df["Exactly same code as in the lookup-table?"].mean()
print(f"Exact success rate {answer_mean}")
results_df.to_csv(output_eval_f, index=False)

# assess response shortlist for exact matches
answers = []
for gold, prediction in zip(results_df["Correct answer"].to_list(), results_df["System shortlist"].to_list()):
    answer = assess_response_shortlist(gold, prediction)
    answers.append(answer)
results_df["Lookup-table answer contained in options?"] = answers
results_df["Lookup-table answer contained in options?"] = results_df["Lookup-table answer contained in options?"].map({'True': True, 'False': False})
answer_mean = results_df["Lookup-table answer contained in options?"].mean()
print(f"Shortlist exact success rate {answer_mean}")
results_df.to_csv(output_shortlist_eval_f, index=False)
results_df

LLM guess for the CT: Systolic blood pressure
Systolic arterial pressure
Systolic blood pressure by manual measurement
Systolic blood pressure by automated measurement
Systolic blood pressure by device
Systolic blood pressure by cuff
Systolic blood pressure by sphygmomanometer
Systolic blood pressure measurement
Systolic pressure
Arterial systolic pressure
Blood pressure systolic
Systolic BP
Brachial systolic pressure
Peripheral systolic pressure
Systolic arterial blood pressure
LLM guess for the CT: Blood pressure diastolic
Diastolic blood pressure
Blood pressure diastolic by manual
Blood pressure diastolic by automated reading
Diastolic blood pressure by automated oscillometric
Diastolic blood pressure by auscultation
Blood pressure diastolic by arterial line
Diastolic blood pressure by cuff
Blood pressure.diastolic
Diastolic blood pressure by Doppler
Blood pressure diastolic supine
Blood pressure diastolic sitting
Blood pressure diastolic standing
Diastolic blood pressure by sphygmo

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2126577089.py:99: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Based on the Indonesian healthcare context and the abbreviation "gdp", here are the most probable matching LOINC long names:

Glucose [Mass/volume] in Blood
Glucose [Mass/volume] in Serum or Plasma
Glucose [Mass/volume] in Capillary blood
Glucose [Mass/volume] in Venous blood
Glucose [Mass/volume] in Arterial blood
Glucose [Moles/volume] in Blood
Glucose [Moles/volume] in Serum or Plasma
Glucose [Mass/volume] in Whole blood
Glucose [Presence] in Blood
Glucose [Mass/volume] in Blood by Glucometer
Glucose [Mass/volume] in Plasma
Glucose measurement device panel
Glucose [Mass/volume] in Urine
Glucose [Presence] in Urine
Glucose [Mass/volume] in Body fluid
LLM guess for the CT: Based on the Indonesian healthcare context and the abbreviation "gd2pp" which likely refers to "Gula Darah 2 Jam Post Prandial" (Blood Sugar 2 Hours Post Prandial), here are the most relevant LOINC long names:

Glucose 2 hours post meal
Glucose [Mass/volume] in Blood 2 hours post meal
Glucose 2

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2126577089.py:99: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Based on the Indonesian healthcare context and the abbreviation "gds" likely referring to "Gula Darah Sewaktu" (random blood glucose), here are the most relevant LOINC long names:

Glucose [Mass/volume] in Blood
Glucose [Mass/volume] in Serum or Plasma
Glucose [Mass/volume] in Capillary blood
Glucose [Mass/volume] in Whole blood
Glucose [Mass/volume] in Blood by Glucometer
Glucose [Moles/volume] in Blood
Glucose [Moles/volume] in Serum or Plasma
Glucose [Mass/volume] in Venous blood
Glucose [Presence] in Blood
Glucose [Mass/volume] in Arterial blood
Glucose [Mass/volume] in Mixed venous blood
Glucose measurement device panel
Glucose [Mass/volume] in Blood by Analysis method
Glucose [Mass/volume] in Serum/Plasma specimen
Glucose [Moles/volume] in Whole blood


...

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2126577089.py:99: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Based on the Indonesian healthcare context and the abbreviation "muac", here are the most relevant LOINC long names:

Mid-upper arm circumference
Arm circumference
Upper arm circumference
Mid-upper arm circumference by Tape measure
Arm circumference mid upper arm
Mid-upper arm circumference protocol
Mid-upper arm circumference percentile
Mid-upper arm circumference Z score
LLM guess for the CT: Tobacco smoking status
Smoking history
History of tobacco use
Tobacco use history
Smoking status
History of smoking
Tobacco use status
Smoking behavior
Tobacco use pattern
Smoking history and current use
Current smoking status
Smoking history assessment
Tobacco use screening
Smoking pattern
Nicotine use history
LLM guess for the CT: Here are relevant LOINC long names for that alcohol consumption query:

Alcoholic drinks per week
Number of alcoholic drinks per week
Alcoholic beverages per week
Alcohol intake per week
Drinks alcohol per week
Weekly alcohol consumption
Alcohol

/var/folders/y3/tg4ts42x7bl1ty6zww9fx4bh0000gn/T/ipykernel_21843/2126577089.py:99: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  guess_keyword_search_results = df[df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())].LONG_COMMON_NAME.to_list()


LLM guess for the CT: Based on the Indonesian healthcare context and the concept of current contraceptive use, here are the most relevant LOINC long names:

Contraceptive method current
Current birth control method
Contraceptive method in use
Birth control method currently used
Contraceptive method currently used
Family planning method current
Current contraception status
Contraceptive method status
Birth control status
Current family planning method
Contraceptive use status
LLM guess for the CT: Drug use behavior
Drug use pattern
Drug use history
Non-prescription drug use
Over-the-counter drug use
Self-medication behavior
Drug use assessment
Drug use screening
Drug consumption behavior
Drug use status
Medication use pattern
Self-treatment history
Non-prescribed medication use
Drug taking behavior
Medication consumption pattern
LLM guess for the CT: Based on the context of patient healthcare records, here are the most relevant LOINC long names that could match "how_many_children":

Num

,Input field,Correct answer,System shortlist,System answer,Exactly same code as in the lookup-table?,Lookup-table answer contained in options?
0,systolic_blood_pressure_mmhg,Systolic blood pressure;8480-6,Systolic blood pressure;8480-6\nSystolic blood...,Systolic blood pressure;8480-6,True,True
1,diastolic_blood_pressure_mmhg,Diastolic blood pressure;8462-4,Diastolic blood pressure;8462-4\nDiastolic blo...,Diastolic blood pressure;8462-4,True,True
2,temperature,Body temperature;8310-5,"Based on the source term ""temperature"" and the...",Body temperature;8310-5,True,True
3,pulse_bpm,Heart rate;8867-4,"Based on the source term ""pulse_bpm"" and the c...",Heart rate;8867-4,True,True
4,respiration_rate_rpm,Respiratory rate;9279-1,Respiratory rate; 9279-1\nRespiratory rate 1 h...,Respiratory rate;9279-1,True,True
5,avpu_score,Level of responsiveness;67775-7,"Based on the source term ""avpu_score"" and the ...",Level of consciousness;80288-4,False,True
6,height,Body height;8302-2,Body height Measured;3137-7\nBody height State...,Body height Measured;3137-7,False,True
7,weight,Body weight;29463-7,Body weight Measured;3141-9\nBody weight;29463...,Body weight Measured;3141-9,False,True
8,head,Physical findings of Head Narrative;10199-8,"Based on the source term ""head"" from a doctor'...",Physical findings of Head Narrative;10199-8,True,True
9,eyes,Physical findings of Eye Narrative;10199-8,"Based on the source term ""eyes"" in a doctor's ...",Physical findings of Eye;8699-1,False,False


In [32]:
# assess if text in shortlist

assess_system_shortlist_prompt = """
You are an assistant for checking whether a target Clinical Terminology entry can be found in a list of candidate entries.
The entry is found only if the full entry, including its exact name and code are contained in the list.
ONLY output True or False with no further explanation.
"""

assess_shortlist_prompt = """
Target term to look for: "{gold}"
List of candidate entries: "{prediction}"
"""

def assess_response_shortlist(gold, prediction):
    """Use Claude to assess correct answer and prediction pair."""

    assess_shortlist_prompt_formatted = assess_shortlist_prompt.format(gold=gold, prediction=prediction)
    message = client.messages.create(
        model= "claude-3-5-sonnet-20241022", #"claude-3-5-sonnet-20241022", #claude-3-5-haiku-20241022
        max_tokens=500,
        temperature=0,
        system=assess_system_shortlist_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": assess_shortlist_prompt_formatted
                    }
                ]
            }
        ]
    )
    assess_response = message.content[0].text

    return assess_response

In [ ]:
results_df["Lookup-table answer contained in options?"].value_counts()

clean up selected pipeline

In [ ]:
v1

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain.chains import TransformChain, SimpleSequentialChain
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict
import json
import pandas as pd
from embeddings.embeddings import SearchResult

# Default prompts
DEFAULT_EXPANSION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant for matching terminology between health record systems. 
    You will be given a source text from one health record system, which might be in another language. 
    Output up to 15 of the most probable equivalent LOINC clinical terminology names (long names only, no codes). 
    Only include terms that match the specificity level of the input - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Add each result on a new line without numbering."""),
    ("user", """{general_info}
    
    Source text for which to output LOINC terms: "{input_text}"
    {specific_info}""")
])

DEFAULT_SHORTLIST_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assisstant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. 
    Select 10 target LOINC terms that reflect the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])

DEFAULT_FINAL_SELECTION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assisstant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. 
    Select just ONE target LOINC term that reflects the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])

class LoincMapper:
    def __init__(self, 
                 anthropic_api_key: str, 
                 loinc_store, 
                 loinc_df: pd.DataFrame,
                 expansion_prompt: ChatPromptTemplate = DEFAULT_EXPANSION_PROMPT,
                 shortlist_prompt: ChatPromptTemplate = DEFAULT_SHORTLIST_PROMPT,
                 final_selection_prompt: ChatPromptTemplate = DEFAULT_FINAL_SELECTION_PROMPT):
        """
        Initialize the LOINC mapper with custom or default prompts
        
        Args:
            anthropic_api_key: API key for Anthropic
            loinc_store: Vector store for LOINC terms
            loinc_df: DataFrame containing LOINC terms
            expansion_prompt: Prompt for expanding input terms
            shortlist_prompt: Prompt for selecting shortlist
            final_selection_prompt: Prompt for final selection
        """
        self.llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", 
                                anthropic_api_key=anthropic_api_key,
                                temperature=0)
        self.loinc_store = loinc_store
        self.loinc_df = loinc_df
        self.loinc_num_dict = dict(zip(loinc_df.LONG_COMMON_NAME.to_list(), 
                                     loinc_df.LOINC_NUM.to_list()))
        self.expansion_prompt = expansion_prompt
        self.shortlist_prompt = shortlist_prompt
        self.final_selection_prompt = final_selection_prompt
        
    def _create_synonym_expansion_chain(self):
        """Creates chain for expanding input term to potential LOINC matches"""
        return self.expansion_prompt | self.llm | StrOutputParser()

    def _create_search_chain(self):
        """Creates chain for searching LOINC database with expanded terms"""
        def search_loinc(inputs: dict) -> dict:
            guesses = inputs["expanded_terms"].split("\n")
            
            # Vector search
            vector_results = []
            for guess in guesses:
                results = self.loinc_store.search(guess, search_kwargs={"k": 10})
                vector_results.extend(results)
            
            # Keyword search
            keyword_results = []
            for guess in guesses:
                matches = self.loinc_df[
                    self.loinc_df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())
                ].LONG_COMMON_NAME.to_list()
                
                if len(matches) > 100:
                    matches = sorted(matches, key=len)[:100]
                    
                keyword_results.extend([
                    SearchResult(
                        text=json.dumps({"LONG_COMMON_NAME": s}),
                        metadata={},
                        score=None
                    ) for s in matches
                ])
            
            # Add LOINC codes to keyword results
            keyword_results = [
                SearchResult(
                    text=json.dumps({
                        'LOINC_NUM': self.loinc_num_dict.get(
                            json.loads(r.text)['LONG_COMMON_NAME']
                        ),
                        **json.loads(r.text)
                    }),
                    metadata=r.metadata,
                    score=r.score
                ) for r in keyword_results
            ]
            
            combined_results = keyword_results + vector_results
            return {"search_results": combined_results}
        
        return TransformChain(
            input_variables=["expanded_terms"],
            output_variables=["search_results"],
            transform=search_loinc
        )

    def _create_shortlist_selection_chain(self):
        """Creates chain for selecting shortlist from search results"""
        return self.shortlist_prompt | self.llm | StrOutputParser()

    def _create_final_selection_chain(self):
        """Creates chain for selecting final term from shortlist"""
        return self.final_selection_prompt | self.llm | StrOutputParser()

    def create_mapping_chain(self):
        """Creates the full LOINC mapping chain"""
        # Create individual chains
        expand_chain = self._create_synonym_expansion_chain()
        search_chain = self._create_search_chain()
        shortlist_chain = self._create_shortlist_selection_chain()
        final_selection_chain = self._create_final_selection_chain()
        
        # Define the full chain
        chain = (
            {
                "expanded_terms": expand_chain,
                "input_text": RunnablePassthrough(),
                "general_info": RunnablePassthrough(),
                "specific_info": RunnablePassthrough()
            }
            | search_chain
            | {
                "shortlist": shortlist_chain,
                "input_text": RunnablePassthrough(),
                "expanded_terms": RunnablePassthrough(),
                "search_results": RunnablePassthrough(),
                "general_info": RunnablePassthrough(),
                "specific_info": RunnablePassthrough()
            }
            | {
                "final_selection": final_selection_chain,
                "shortlist": RunnablePassthrough()
            }
        )
        
        return chain

# Usage example:
"""
# Define custom prompts (optional)
custom_expansion_prompt = ChatPromptTemplate.from_messages([
    ("system", "Your custom system message"),
    ("user", "Your custom user message template")
])

# Initialize mapper with custom or default prompts
mapper = LoincMapper(
    anthropic_api_key=ANTHROPIC_API_KEY,
    loinc_store=loinc_store,
    loinc_df=df,
    expansion_prompt=custom_expansion_prompt  # Optional
)

# Create and run chain
mapping_chain = mapper.create_mapping_chain()
result = mapping_chain.invoke({
    "input_text": field,
    "general_info": "The input term is from the Indonesian Satusehat patient healthcare records system.",
    "specific_info": "This is a doctor's narrative field" if "narrative" in gold_label.lower() else ""
})

print(f"Final selection: {result['final_selection']}")
print(f"Shortlist: {result['shortlist']}")
"""

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
import json
import pandas as pd

# Default prompts
DEFAULT_EXPANSION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant for matching terminology between health record systems. 
    You will be given a source text from one health record system, which might be in another language. 
    Output up to 15 of the most probable equivalent LOINC clinical terminology names (long names only, no codes). 
    Only include terms that match the specificity level of the input - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Add each result on a new line without numbering."""),
    ("user", """{general_info}
    
    Source text for which to output LOINC terms: "{input_text}"
    {specific_info}""")
])

DEFAULT_SHORTLIST_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assisstant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. 
    Select 10 target LOINC terms that reflect the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])

DEFAULT_FINAL_SELECTION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assisstant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. 
    Select just ONE target LOINC term that reflects the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])

class LoincMapper:
    def __init__(self, 
                 anthropic_api_key: str, 
                 loinc_store,  # Vector store for LOINC terms
                 loinc_df: pd.DataFrame):
        """Initialize the LOINC mapper."""
        self.llm = ChatAnthropic(
            model="claude-3-sonnet-20240229",
            anthropic_api_key=anthropic_api_key,
            # temperature=0 # TODO
        )
        self.loinc_store = loinc_store
        self.loinc_df = loinc_df
        self.loinc_num_dict = dict(zip(loinc_df.LONG_COMMON_NAME, loinc_df.LOINC_NUM))
        
    def _search_loinc(self, inputs: dict) -> dict:
        """Search LOINC database with expanded terms."""
        guesses = inputs["expanded_terms"].split("\n")
        
        # Vector search
        vector_results = []
        for guess in guesses:
            results = self.loinc_store.search(guess, search_kwargs={"k": 10})
            vector_results.extend(results)
        
        # Keyword search 
        keyword_results = []
        for guess in guesses:
            matches = self.loinc_df[
                self.loinc_df["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())
            ].LONG_COMMON_NAME.to_list()
            
            if len(matches) > 100:
                matches = matches[:100]
                
            keyword_results.extend([{
                "text": json.dumps({
                    "LONG_COMMON_NAME": s,
                    "LOINC_NUM": self.loinc_num_dict.get(s)
                }),
                "metadata": {},
                "score": None
            } for s in matches])
        
        return {"search_results": keyword_results + vector_results}

    def create_mapping_chain(self):
        """Create the LOINC mapping chain."""
        # Create expansion chain
        expand_chain = (
            DEFAULT_EXPANSION_PROMPT 
            | self.llm 
            | StrOutputParser()
        )
        
        # Create search chain
        search_chain = RunnablePassthrough() | self._search_loinc
        
        # Create shortlist chain
        shortlist_chain = (
            DEFAULT_SHORTLIST_PROMPT 
            | self.llm 
            | StrOutputParser()
        )
        
        # Create final selection chain
        final_selection_chain = (
            DEFAULT_FINAL_SELECTION_PROMPT 
            | self.llm 
            | StrOutputParser()
        )
        
        # Combine chains
        chain = (
            {
                "expanded_terms": expand_chain,
                "input_text": RunnablePassthrough(),
                "general_info": RunnablePassthrough(),
                "specific_info": RunnablePassthrough()
            }
            | search_chain
            | {
                "shortlist": shortlist_chain,
                "input_text": RunnablePassthrough(),
                "expanded_terms": RunnablePassthrough(),
                "search_results": RunnablePassthrough(),
                "general_info": RunnablePassthrough(),
                "specific_info": RunnablePassthrough()
            }
            | {
                "final_selection": final_selection_chain,
                "shortlist": RunnablePassthrough()
            }
        )
        
        return chain

    def map_term(self, field: str, general_info: str, specific_info: str = "") -> dict:
        """Map a term to LOINC.
        
        Args:
            field: The source term to map
            general_info: General context about the source system
            specific_info: Additional specific context about this term
        """
        chain = self.create_mapping_chain()
        result = chain.invoke({
            "input_text": field,
            "general_info": general_info,
            "specific_info": specific_info
        })
        return result

# Usage example:
"""
# Initialize mapper
mapper = LoincMapper(
    anthropic_api_key="your-api-key",
    loinc_store=your_vector_store,
    loinc_df=your_loinc_dataframe
)

# Map a term
result = mapper.map_term(
    field="Blood glucose",
    general_info="The input term is from the Indonesian Satusehat patient healthcare records system",
    specific_info="This term appears in the laboratory results section"
)

print(f"Final selection: {result['final_selection']}")
print(f"Shortlist: {result['shortlist']}")
"""

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
import json
import pandas as pd

linear chaining

In [26]:

EXPANSION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant for matching terminology between health record systems. 
    You will be given a source text from one health record system, which might be in another language. 
    Output up to 15 of the most probable equivalent LOINC clinical terminology names (long names only, no codes). 
    Only include terms that match the specificity level of the input - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Add each result on a new line without numbering."""),
    ("user", """{general_info}
    
    Source text for which to output LOINC terms: "{input_text}"
    {specific_info}""")
])

SHORTLIST_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. Select 10 target LOINC terms that reflect the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just a list of the LONG_COMMON_NAMEs and the LOINC_NUMBERs separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])

FINAL_SELECTION_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an assisstant for matching terminology between health record systems. 
    You will be given an entry from a source health record system, and a list of possible target LOINC terms. Select just ONE target LOINC term that reflects the original source term best. 
    To help you, you will be given a list of likely entry names to look out for. Additional contextual information may also be given.
    Only consider terms that match the specificity level of the source text - do not add qualifiers or measurement methods unless they are explicitly stated in the input or accompanying context.
    Give your response as just the LONG_COMMON_NAME and the LOINC_NUMBER separated by a semicolon."""),
    ("user", """{general_info}
    Original source term for which to look for a LOINC entry: {input_text}
    {specific_info}
    The correct target LOINC entry text is likely to look like one of these: {expanded_terms}
    LOINC entries to select from: "{search_results}" """)
])



class VocabMapper:
    def __init__(self, 
                 anthropic_api_key: str, 
                 vectorstore,
                 dataset: pd.DataFrame):
        """Initialize the vocab mapper."""
        self.llm = ChatAnthropic(
            model="claude-3-sonnet-20240229",
            anthropic_api_key=anthropic_api_key
        )
        self.vectorstore = vectorstore
        self.dataset = dataset
        self.loinc_num_dict = dict(zip(dataset.LONG_COMMON_NAME, dataset.LOINC_NUM))

    def get_expanded_terms(self, input_text: str, general_info: str, specific_info: str) -> str:
        """Step 1: Get expanded list of possible terms."""
        chain = EXPANSION_PROMPT | self.llm | StrOutputParser()
        expanded_terms = chain.invoke({
            "input_text": input_text,
            "general_info": general_info,
            "specific_info": specific_info
        })
        return expanded_terms

    def search_database(self, expanded_terms: str) -> list:
        """Step 2: Search the database for the expanded terms."""
        # Vector search
        vector_results = []
        for guess in expanded_terms.split("\n"):
            results = self.vectorstore.search(guess, search_kwargs={"k": 10})
            vector_results.extend(results)
        
        # Keyword search
        keyword_results = []
        for guess in expanded_terms.split("\n"):
            matches = self.dataset[
                self.dataset["LONG_COMMON_NAME"].str.lower().str.contains(guess.lower())
            ].LONG_COMMON_NAME.to_list()[:100]
                
            keyword_results.extend([{
                "text": json.dumps({
                    "LONG_COMMON_NAME": s,
                    "LOINC_NUM": self.loinc_num_dict.get(s)
                }),
                "metadata": {},
                "score": None
            } for s in matches])
        
        return keyword_results + vector_results

    def get_shortlist(self, input_text: str, general_info: str, specific_info: str, 
                     expanded_terms: str, search_results: list) -> str:
        """Step 3: Get a shortlist of the best matches."""
        chain = SHORTLIST_PROMPT | self.llm | StrOutputParser()
        shortlist = chain.invoke({
            "input_text": input_text,
            "general_info": general_info,
            "specific_info": specific_info,
            "expanded_terms": expanded_terms,
            "search_results": search_results
        })
        return shortlist

    def get_final_selection(self, input_text: str, general_info: str, specific_info: str,
                          expanded_terms: str, search_results: list, shortlist: str) -> str:
        """Step 4: Get the best match."""
        chain = FINAL_SELECTION_PROMPT | self.llm | StrOutputParser()
        final_selection = chain.invoke({
            "input_text": input_text,
            "general_info": general_info,
            "specific_info": specific_info,
            "expanded_terms": expanded_terms,
            "search_results": search_results
        })
        return final_selection

    def map_term(self, input_term: str, general_info: str, specific_info: str = "") -> dict:
        """Map an input term using the target dataset."""
        # Step 1: Get expanded terms
        expanded_terms = self.get_expanded_terms(input_term, general_info, specific_info)
        
        # Step 2: Search database
        search_results = self.search_database(expanded_terms)
        
        # Step 3: Get shortlist of best terms
        shortlist = self.get_shortlist(
            input_term, general_info, specific_info,
            expanded_terms, search_results
        )
        
        # Step 4: Select the best term (from the full search results)
        final_selection = self.get_final_selection(
            input_term, general_info, specific_info,
            expanded_terms, search_results, shortlist
        )
        
        # Return all results
        return {
            "expanded_terms": expanded_terms,
            # "search_results": search_results, # large - can add back in for debugging
            "shortlist": shortlist,
            "final_selection": final_selection
        }



In [7]:
from datasets import load_dataset
from langchain_community.document_loaders import DataFrameLoader

loinc_df = load_dataset("awacke1/LOINC-Clinical-Terminology")
loinc_df = pd.DataFrame(loinc_df['train'])

In [8]:
from dotenv import load_dotenv

load_dotenv(override=True)

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")


In [ ]:
mapper = VocabMapper(
    anthropic_api_key=ANTHROPIC_API_KEY,
    vectorstore=loinc_store,
    dataset=loinc_df
)

result = mapper.map_term(
    input_term="systolic_blood_pressure_mmhge",
    general_info="The input term is from the Indonesian Satusehat patient healthcare records system.",
    specific_info=""
)

result

{'expanded_terms': 'Systolic blood pressure\nSystolic arterial BP\nSystolic BP\nBP systolic\nBlood pressure systolic\nSystolic pressure',
 'shortlist': 'Here are my top 10 LOINC term matches for the source term "systolic_blood_pressure_mmhg":\n\n8480-6;Systolic blood pressure\n8508-4;Brachial artery Systolic blood pressure  \n76534-7;Systolic blood pressure by Noninvasive\n75997-7;Systolic blood pressure by Continuous non-invasive monitoring\n8459-0;Systolic blood pressure--sitting\n8460-8;Systolic blood pressure--standing  \n8461-6;Systolic blood pressure--supine\n76215-3;Invasive Systolic blood pressure\n8484-8;Systolic blood pressure 12 hour maximum\n8494-7;Systolic blood pressure 12 hour minimum',
 'final_selection': 'Based on the original source term "systolic_blood_pressure_mmhge" and the list of LOINC entries provided, the best match seems to be:\n\nSystolic blood pressure; 8480-6\n\nThis maps to the LONG_COMMON_NAME "Systolic blood pressure" and the LOINC_NUMBER 8480-6. It is a

In [18]:
from typing import List, Dict

In [ ]:
def process_inputs(input_data: List[Dict]) -> List[Dict]:
    """Process inputs to map one by one."""
    results = []
    for row in input_data:
        result = mapper.map_term(
            input_term=row['input_term'],
            general_info=row.get('general_info', ''),
            specific_info=row.get('specific_info', '')
        )
        results.append({
            'input': row,
            'mapping': result
        })
    return results

In [27]:
mapper = VocabMapper(
    anthropic_api_key=ANTHROPIC_API_KEY,
    vectorstore=loinc_store,
    dataset=loinc_df
)

inputs = [{'input_term':'systolic_blood_pressure_mmhge', 'general_info':'', 'specific_info':''}, {'input_term':'diastolic_blood_pressure_mmhg', 'general_info':'', 'specific_info':''}]

results = process_inputs(inputs)
results

[{'input': {'input_term': 'systolic_blood_pressure_mmhge',
   'general_info': '',
   'specific_info': ''},
  'mapping': {'expanded_terms': 'Systolic blood pressure',
   'shortlist': 'Systolic blood pressure; 8480-6\nSystolic blood pressure--sitting; 8459-0  \nSystolic blood pressure--standing; 8460-8\nPeak systolic blood pressure --during aorta stenosis maximum velocity measurement; 24371-7\nSystolic blood pressure--supine; 8461-6\nSystolic blood pressure 12 hour minimum; 8494-7\nSystolic blood pressure 12 hour maximum; 8484-8\nInvasive Systolic blood pressure; 76215-3\nSystolic blood pressure 8 hour minimum; 8492-1\nBrachial artery Systolic blood pressure; 8508-4',
   'final_selection': 'Systolic blood pressure; 8480-6'}},
 {'input': {'input_term': 'diastolic_blood_pressure_mmhg',
   'general_info': '',
   'specific_info': ''},
  'mapping': {'expanded_terms': 'Diastolic blood pressure\nDiastolic blood pressure Systolic blood pressure and diastolic blood pressure\nDiastolic blood press

In [25]:
inputs = [{'input_term':'systolic_blood_pressure_mmhge'}, {'input_term':'diastolic_blood_pressure_mmhg'}]

results = process_inputs(inputs)
results

[{'input': {'input_term': 'systolic_blood_pressure_mmhge'},
  'mapping': {'expanded_terms': 'Systolic blood pressure\nSystolic blood pressure -- expiration\nSystolic blood pressure -- inspiration\nSystolic blood pressure measurement\nSystolic blood pressure -- mean\nSystolic blood pressure -- maximum\nSystolic blood pressure -- minimum\nSystolic blood pressure from cubitalis\nSystolic blood pressure from dorsalis pedis\nSystolic blood pressure from popliteal\nSystolic blood pressure from posterior tibial\nSystolic blood pressure from radial\nSystolic blood pressure from tibialis posterior',
   'shortlist': 'Here are 10 LOINC codes and their corresponding long common names that best match the original source term "systolic_blood_pressure_mmhge":\n\n1. 8480-6; Systolic blood pressure\n2. 8451-7; Systolic blood pressure--inspiration  \n3. 8450-9; Systolic blood pressure--expiration\n4. 8459-0; Systolic blood pressure--sitting\n5. 8460-8; Systolic blood pressure--standing\n6. 8461-6; Systo

In [ ]:
# evaluate new clean version

inputs = [{'input_term':input_term, 'general_info':'The input term is from the Indonesian Satusehat patient healthcare records system.', 'specific_info':''} for input_term in fields]
for input, gold in zip(inputs, gold_labels):
    if 'narrative' in gold.lower():
        input['specific_info'] = "This is a doctor's narrative field"
results = process_inputs(inputs)
results_df = pd.DataFrame(results)
results_df['Correct answer'] = gold_labels


[{'input_term': 'systolic_blood_pressure_mmhg',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'input_term': 'diastolic_blood_pressure_mmhg',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'input_term': 'temperature',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'input_term': 'pulse_bpm',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'input_term': 'respiration_rate_rpm',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'input_term': 'avpu_score',
  'general_info': 'The input term is from the Indonesian Satusehat patient healthcare records system.',
  'specific_info': ''},
 {'in

In [ ]:
# assess selected responses for exact matches
answers = []
for gold, prediction in zip(results_df["Correct answer"].to_list(), results_df["System answer"].to_list()):
    answer = assess_response(gold, prediction)
    answers.append(answer)
results_df["Exactly same code as in the lookup-table?"] = answers
results_df["Exactly same code as in the lookup-table?"] = results_df["Exactly same code as in the lookup-table?"].map({'True': True, 'False': False})
answer_mean = results_df["Exactly same code as in the lookup-table?"].mean()
print(f"Exact success rate {answer_mean}")
results_df.to_csv(output_eval_f, index=False)

# assess response shortlist for exact matches
answers = []
for gold, prediction in zip(results_df["Correct answer"].to_list(), results_df["System shortlist"].to_list()):
    answer = assess_response_shortlist(gold, prediction)
    answers.append(answer)
results_df["Lookup-table answer contained in options?"] = answers
results_df["Lookup-table answer contained in options?"] = results_df["Lookup-table answer contained in options?"].map({'True': True, 'False': False})
answer_mean = results_df["Lookup-table answer contained in options?"].mean()
print(f"Shortlist exact success rate {answer_mean}")
results_df.to_csv(output_shortlist_eval_f, index=False)
results_df